# Module 5：Attention 與 Transformer

## 學習目標

1. 理解 Attention 機制的動機與原理
2. 掌握 Self-Attention 的 Q, K, V 概念
3. 理解 Multi-Head Attention 的優勢
4. 學會 Positional Encoding 的作用
5. 完整實作 Transformer Encoder
6. 了解 BERT 和 GPT 的核心思想

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import math

# 設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(42)
plt.rcParams['figure.figsize'] = (12, 4)

---

## Part 1：Attention 的動機

### 1.1 RNN 的瓶頸

**問題：** Seq2Seq（如機器翻譯）把整個輸入壓縮成一個固定向量

```
Encoder:  [I] → [love] → [you] → [context vector]
                                        ↓
Decoder:                           [我] → [愛] → [你]
```

**瓶頸：** context vector 容量有限，長句子資訊會丟失

### 1.2 Attention 的解決方案

**想法：** Decoder 在生成每個詞時，可以「回頭看」Encoder 的所有輸出，決定要關注哪些部分

```
生成「愛」時：
  - 看 "I"    → attention weight: 0.1
  - 看 "love" → attention weight: 0.8  ← 重點關注！
  - 看 "you"  → attention weight: 0.1
```

In [ ]:
# Attention 的核心概念

print("""
Attention 三要素：

1. Query (Q)：「我想找什麼」- 當前要生成的位置
2. Key (K)：「這裡有什麼」- 每個可以被關注的位置
3. Value (V)：「這裡的內容」- 實際要取出的資訊

計算過程：
┌─────────────────────────────────────────────────────────┐
│  1. 計算相似度：score = Q · K^T                          │
│  2. 正規化：attention_weights = softmax(score)           │
│  3. 加權求和：output = attention_weights · V             │
└─────────────────────────────────────────────────────────┘

直覺比喻：圖書館查書
  - Query：你想找的主題（如「深度學習」）
  - Key：每本書的標籤/索引
  - Value：每本書的實際內容
  - 過程：用 Query 和每個 Key 比對，找出最相關的書，取出內容
""")

In [ ]:
# 簡單的 Attention 實現

def simple_attention(query, keys, values):
    """
    最簡單的 Attention
    query: (batch, d_k)
    keys: (batch, seq_len, d_k)
    values: (batch, seq_len, d_v)
    """
    # 計算相似度 (用內積)
    # query: (batch, d_k) -> (batch, 1, d_k)
    scores = torch.bmm(query.unsqueeze(1), keys.transpose(1, 2))  # (batch, 1, seq_len)
    scores = scores.squeeze(1)  # (batch, seq_len)
    
    # Softmax 得到權重
    attention_weights = F.softmax(scores, dim=-1)  # (batch, seq_len)
    
    # 加權求和
    # attention_weights: (batch, seq_len) -> (batch, 1, seq_len)
    output = torch.bmm(attention_weights.unsqueeze(1), values)  # (batch, 1, d_v)
    output = output.squeeze(1)  # (batch, d_v)
    
    return output, attention_weights

# 測試
batch_size = 2
seq_len = 4
d_k = 8
d_v = 8

query = torch.randn(batch_size, d_k)
keys = torch.randn(batch_size, seq_len, d_k)
values = torch.randn(batch_size, seq_len, d_v)

output, attn_weights = simple_attention(query, keys, values)

print(f"Query shape: {query.shape}")
print(f"Keys shape: {keys.shape}")
print(f"Values shape: {values.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\nAttention weights (sample 0): {attn_weights[0]}")
print(f"Sum of weights: {attn_weights[0].sum():.4f}")

---

## Part 2：Self-Attention

### 2.1 什麼是 Self-Attention？

**普通 Attention：** Query 來自一個序列，Key/Value 來自另一個序列

**Self-Attention：** Query, Key, Value 都來自**同一個序列**

**效果：** 讓序列中的每個位置都能「看到」其他所有位置

In [ ]:
# Self-Attention 示意

print("""
Self-Attention 範例：

輸入句子："The cat sat on the mat"

當處理 "sat" 這個詞時：
  - Query："sat" 的表示
  - Keys：所有詞 ["The", "cat", "sat", "on", "the", "mat"] 的表示
  - Values：同上

Self-Attention 會計算 "sat" 和每個詞的相關性：
  "The"  →  0.05
  "cat"  →  0.40  ← 「貓」和「坐」相關！
  "sat"  →  0.15
  "on"   →  0.10
  "the"  →  0.05
  "mat"  →  0.25  ← 「坐」在「墊子」上

優點：
1. 任意兩個位置可以直接連接（不用像 RNN 一步步傳遞）
2. 可以並行計算（不需要順序處理）
3. 能捕捉長距離依賴
""")

### 2.2 Scaled Dot-Product Attention

**公式：** 
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

**為什麼要除以 $\sqrt{d_k}$？**
- 當 $d_k$ 很大時，內積值會很大
- 大的值會讓 softmax 輸出接近 one-hot（梯度接近 0）
- 除以 $\sqrt{d_k}$ 讓分數保持在合理範圍

In [ ]:
# Scaled Dot-Product Attention 實現

def scaled_dot_product_attention(query, key, value, mask=None):
    """
    Scaled Dot-Product Attention
    
    query: (batch, seq_len_q, d_k)
    key: (batch, seq_len_k, d_k)
    value: (batch, seq_len_k, d_v)
    mask: (batch, seq_len_q, seq_len_k) or None
    """
    d_k = query.size(-1)
    
    # 計算注意力分數
    # (batch, seq_q, d_k) @ (batch, d_k, seq_k) -> (batch, seq_q, seq_k)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    
    # 應用 mask（如果有）
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))
    
    # Softmax
    attention_weights = F.softmax(scores, dim=-1)
    
    # 加權求和
    output = torch.matmul(attention_weights, value)
    
    return output, attention_weights

# 測試
batch_size = 2
seq_len = 4
d_model = 8

# Self-Attention：Q, K, V 來自同一個序列
x = torch.randn(batch_size, seq_len, d_model)
output, attn_weights = scaled_dot_product_attention(x, x, x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")

In [ ]:
# 視覺化 Attention Weights

# 模擬一個句子的 attention
words = ["The", "cat", "sat", "mat"]
seq_len = len(words)

torch.manual_seed(42)
x = torch.randn(1, seq_len, 16)
_, attn = scaled_dot_product_attention(x, x, x)
attn = attn.squeeze(0).numpy()

plt.figure(figsize=(8, 6))
plt.imshow(attn, cmap='Blues')
plt.colorbar(label='Attention Weight')
plt.xticks(range(seq_len), words)
plt.yticks(range(seq_len), words)
plt.xlabel('Key (被關注的位置)')
plt.ylabel('Query (當前位置)')
plt.title('Self-Attention Weights')

# 標註數值
for i in range(seq_len):
    for j in range(seq_len):
        plt.text(j, i, f'{attn[i, j]:.2f}', ha='center', va='center')

plt.tight_layout()
plt.show()

### 2.3 Attention Mask

**用途：**
1. **Padding Mask**：忽略 padding 位置
2. **Causal Mask**：防止「看到未來」（用於 GPT 等自回歸模型）

In [ ]:
# Causal Mask（下三角矩陣）

def create_causal_mask(seq_len):
    """建立因果遮罩，防止看到未來的位置"""
    mask = torch.tril(torch.ones(seq_len, seq_len))
    return mask

seq_len = 5
causal_mask = create_causal_mask(seq_len)

print("Causal Mask:")
print(causal_mask)
print("\n1 = 可以看, 0 = 不能看")
print("每個位置只能看到自己和之前的位置")

# 視覺化
plt.figure(figsize=(6, 5))
plt.imshow(causal_mask, cmap='Greens')
plt.colorbar()
plt.xlabel('Key position')
plt.ylabel('Query position')
plt.title('Causal Mask (for autoregressive models)')
plt.xticks(range(seq_len))
plt.yticks(range(seq_len))
plt.show()

In [ ]:
# 有 Mask 和沒有 Mask 的差異

torch.manual_seed(42)
x = torch.randn(1, 5, 8)

# 無 mask
_, attn_no_mask = scaled_dot_product_attention(x, x, x, mask=None)

# 有 causal mask
mask = create_causal_mask(5).unsqueeze(0)
_, attn_with_mask = scaled_dot_product_attention(x, x, x, mask=mask)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(attn_no_mask.squeeze().numpy(), cmap='Blues')
axes[0].set_title('Without Mask (Bidirectional)')
axes[0].set_xlabel('Key')
axes[0].set_ylabel('Query')

axes[1].imshow(attn_with_mask.squeeze().numpy(), cmap='Blues')
axes[1].set_title('With Causal Mask (Unidirectional)')
axes[1].set_xlabel('Key')
axes[1].set_ylabel('Query')

plt.tight_layout()
plt.show()

print("觀察：有 Causal Mask 時，上三角區域的 attention 權重為 0")

---

## Part 3：Multi-Head Attention

### 3.1 為什麼需要 Multi-Head？

**問題：** 單一的 attention 只能學習一種「關注模式」

**解決方案：** 用多個「頭」(heads) 來捕捉不同類型的關係

- Head 1：可能學習語法關係（主詞-動詞）
- Head 2：可能學習指代關係（代名詞-名詞）
- Head 3：可能學習位置關係（相鄰詞）
- ...

In [ ]:
# Multi-Head Attention 結構

print("""
Multi-Head Attention 結構：

輸入 X (batch, seq_len, d_model)
      │
      ├──→ Linear → Q₁ ──┐
      ├──→ Linear → K₁ ──┼──→ Attention Head 1 ──┐
      ├──→ Linear → V₁ ──┘                       │
      │                                          │
      ├──→ Linear → Q₂ ──┐                       │
      ├──→ Linear → K₂ ──┼──→ Attention Head 2 ──┼──→ Concat ──→ Linear ──→ 輸出
      ├──→ Linear → V₂ ──┘                       │
      │                                          │
      ├──→ Linear → Q₃ ──┐                       │
      ├──→ Linear → K₃ ──┼──→ Attention Head 3 ──┘
      └──→ Linear → V₃ ──┘

數學表示：
  head_i = Attention(Q·W_i^Q, K·W_i^K, V·W_i^V)
  MultiHead(Q, K, V) = Concat(head_1, ..., head_h)·W^O

維度關係：
  d_model = h × d_k  (例如 512 = 8 × 64)
""")

In [ ]:
# Multi-Head Attention 實現

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        
        assert d_model % num_heads == 0, "d_model 必須能被 num_heads 整除"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Q, K, V 的線性變換
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        
        # 輸出的線性變換
        self.W_o = nn.Linear(d_model, d_model)
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        # 線性變換
        Q = self.W_q(query)  # (batch, seq_len, d_model)
        K = self.W_k(key)
        V = self.W_v(value)
        
        # 分成多個頭
        # (batch, seq_len, d_model) -> (batch, seq_len, num_heads, d_k)
        # -> (batch, num_heads, seq_len, d_k)
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Scaled Dot-Product Attention (對每個頭)
        # scores: (batch, num_heads, seq_q, seq_k)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            # mask 需要擴展到 (batch, 1, seq_q, seq_k) 或 (batch, num_heads, seq_q, seq_k)
            if mask.dim() == 3:
                mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attention_weights = F.softmax(scores, dim=-1)
        
        # 加權求和
        context = torch.matmul(attention_weights, V)  # (batch, num_heads, seq_len, d_k)
        
        # 合併多個頭
        # (batch, num_heads, seq_len, d_k) -> (batch, seq_len, num_heads, d_k)
        # -> (batch, seq_len, d_model)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # 最後的線性變換
        output = self.W_o(context)
        
        return output, attention_weights

# 測試
d_model = 64
num_heads = 8
seq_len = 10
batch_size = 2

mha = MultiHeadAttention(d_model, num_heads)
x = torch.randn(batch_size, seq_len, d_model)

output, attn_weights = mha(x, x, x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"  = (batch, num_heads, seq_len_q, seq_len_k)")

In [ ]:
# 視覺化不同 Head 的 Attention

# 取第一個樣本的 attention weights
attn_sample = attn_weights[0].detach().numpy()  # (num_heads, seq_q, seq_k)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i in range(8):
    ax = axes[i]
    im = ax.imshow(attn_sample[i], cmap='Blues')
    ax.set_title(f'Head {i+1}')
    ax.set_xlabel('Key')
    ax.set_ylabel('Query')
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.suptitle('Attention Patterns of Different Heads', fontsize=14)
plt.tight_layout()
plt.show()

print("觀察：不同的 head 學到不同的 attention 模式")

---

## Part 4：Positional Encoding

### 4.1 為什麼需要位置編碼？

**問題：** Self-Attention 是「位置無關」的
- "I love you" 和 "you love I" 經過 attention 會得到相同的結果！

**解決方案：** 加入位置資訊 (Positional Encoding)

### 4.2 正弦餘弦位置編碼

原始 Transformer 使用正弦/餘弦函數：

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

In [ ]:
# Positional Encoding 實現

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # 建立位置編碼矩陣
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # 計算分母
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                            (-math.log(10000.0) / d_model))
        
        # 偶數位置用 sin，奇數位置用 cos
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 加上 batch 維度並註冊為 buffer（不是參數，但會被保存）
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# 測試
pe = PositionalEncoding(d_model=64, max_len=100)
x = torch.zeros(1, 50, 64)  # 全零輸入，方便觀察 PE
output = pe(x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {output.shape}")

In [ ]:
# 視覺化 Positional Encoding

d_model = 64
max_len = 100
pe = PositionalEncoding(d_model, max_len, dropout=0)

# 取得 PE 矩陣
pe_matrix = pe.pe.squeeze().numpy()  # (max_len, d_model)

plt.figure(figsize=(12, 6))
plt.imshow(pe_matrix[:50, :], cmap='RdBu', aspect='auto')
plt.colorbar(label='Value')
plt.xlabel('Embedding Dimension')
plt.ylabel('Position')
plt.title('Positional Encoding Visualization')
plt.tight_layout()
plt.show()

# 顯示幾個維度隨位置的變化
plt.figure(figsize=(12, 4))
positions = np.arange(50)
for dim in [0, 1, 4, 5, 20, 21]:
    plt.plot(positions, pe_matrix[:50, dim], label=f'dim {dim}')
plt.xlabel('Position')
plt.ylabel('Value')
plt.title('Positional Encoding: Different Dimensions')
plt.legend()
plt.grid(True)
plt.show()

print("觀察：")
print("- 低維度的週期短（變化快）")
print("- 高維度的週期長（變化慢）")
print("- 每個位置都有獨特的編碼")

---

## Part 5：Transformer Encoder

### 5.1 Encoder Block 結構

```
輸入
  │
  ├──────────────────────┐
  ↓                      │
Multi-Head Attention     │
  │                      │
  ↓                      │
Add & Norm ←─────────────┘ (殘差連接)
  │
  ├──────────────────────┐
  ↓                      │
Feed Forward Network     │
  │                      │
  ↓                      │
Add & Norm ←─────────────┘ (殘差連接)
  │
  ↓
輸出
```

In [ ]:
# Feed Forward Network

class FeedForward(nn.Module):
    """Position-wise Feed Forward Network"""
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        x = self.linear1(x)
        x = F.gelu(x)  # 原始用 ReLU，現代常用 GELU
        x = self.dropout(x)
        x = self.linear2(x)
        return x

# 測試
ff = FeedForward(d_model=64, d_ff=256)
x = torch.randn(2, 10, 64)
print(f"FeedForward: {x.shape} -> {ff(x).shape}")

In [ ]:
# Transformer Encoder Layer

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Self-Attention with residual connection
        attn_output, attn_weights = self.self_attention(x, x, x, mask)
        x = x + self.dropout1(attn_output)  # 殘差連接
        x = self.norm1(x)  # Layer Norm
        
        # Feed Forward with residual connection
        ff_output = self.feed_forward(x)
        x = x + self.dropout2(ff_output)  # 殘差連接
        x = self.norm2(x)  # Layer Norm
        
        return x, attn_weights

# 測試
encoder_layer = TransformerEncoderLayer(d_model=64, num_heads=8, d_ff=256)
x = torch.randn(2, 10, 64)
output, attn = encoder_layer(x)
print(f"Encoder Layer: {x.shape} -> {output.shape}")

In [ ]:
# 完整的 Transformer Encoder

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, 
                 num_layers, max_len=512, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        
        # Embedding 層
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len, dropout)
        
        # 多層 Encoder
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model)
    
    def forward(self, x, mask=None):
        # x: (batch, seq_len) - 詞索引
        
        # Embedding + Positional Encoding
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        
        # 通過所有 Encoder 層
        attention_weights = []
        for layer in self.layers:
            x, attn = layer(x, mask)
            attention_weights.append(attn)
        
        x = self.norm(x)
        
        return x, attention_weights

# 測試
encoder = TransformerEncoder(
    vocab_size=1000,
    d_model=64,
    num_heads=8,
    d_ff=256,
    num_layers=4,
    max_len=100
)

x = torch.randint(0, 1000, (2, 20))  # (batch, seq_len)
output, attentions = encoder(x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {output.shape}")
print(f"\n層數: {len(attentions)}")
print(f"每層 attention shape: {attentions[0].shape}")
print(f"\n總參數量: {sum(p.numel() for p in encoder.parameters()):,}")

---

## Part 6：實作 - 用 Transformer 做文本分類

In [ ]:
# 建立資料集（使用 Module 4 的情感分析資料）

from collections import Counter

# 正面和負面評論
positive_texts = [
    "This movie is great and amazing",
    "I love this film so much",
    "Excellent acting and wonderful story",
    "Best movie I have ever seen",
    "Fantastic film with great performances",
    "This is absolutely brilliant",
    "Amazing movie highly recommended",
    "Wonderful experience great movie",
    "I really enjoyed this film",
    "Perfect movie loved every moment",
    "Outstanding performance by all actors",
    "This film exceeded my expectations",
    "Beautiful story and amazing visuals",
    "One of the best films ever",
    "Incredible movie must watch",
]

negative_texts = [
    "This movie is terrible and boring",
    "I hate this film completely",
    "Worst movie I have ever seen",
    "Bad acting and terrible story",
    "Complete waste of time",
    "This is absolutely awful",
    "Boring movie do not watch",
    "Terrible experience bad movie",
    "I really hated this film",
    "Worst film avoid at all costs",
    "Disappointing and poorly made",
    "This movie was so bad",
    "Horrible story and bad acting",
    "One of the worst films ever",
    "Awful movie not recommended",
]

all_texts = positive_texts + negative_texts
all_labels = [1] * len(positive_texts) + [0] * len(negative_texts)

# 建立詞彙表
word_counts = Counter()
for text in all_texts:
    word_counts.update(text.lower().split())

vocab = ['<PAD>', '<UNK>'] + [w for w, c in word_counts.most_common()]
word2idx = {w: i for i, w in enumerate(vocab)}
vocab_size = len(vocab)

print(f"詞彙表大小: {vocab_size}")

In [ ]:
# 資料集類別

class TextDataset(Dataset):
    def __init__(self, texts, labels, word2idx, max_len=20):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        words = self.texts[idx].lower().split()
        tokens = [self.word2idx.get(w, 1) for w in words]  # 1 = <UNK>
        
        # Padding
        if len(tokens) < self.max_len:
            tokens = tokens + [0] * (self.max_len - len(tokens))
        else:
            tokens = tokens[:self.max_len]
        
        return {
            'input_ids': torch.tensor(tokens),
            'label': torch.tensor(self.labels[idx])
        }

# 分割資料
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    all_texts, all_labels, test_size=0.2, random_state=42
)

train_dataset = TextDataset(train_texts, train_labels, word2idx)
test_dataset = TextDataset(test_texts, test_labels, word2idx)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

In [ ]:
# Transformer 分類器

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, 
                 num_layers, num_classes, max_len=100, dropout=0.1):
        super().__init__()
        
        self.encoder = TransformerEncoder(
            vocab_size, d_model, num_heads, d_ff, num_layers, max_len, dropout
        )
        
        # 分類頭
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )
    
    def forward(self, x, mask=None):
        # x: (batch, seq_len)
        
        # Encoder
        encoded, attentions = self.encoder(x, mask)
        
        # 取 [CLS] 位置（這裡用平均池化代替）
        # pooled = encoded[:, 0, :]  # 如果有 [CLS] token
        pooled = encoded.mean(dim=1)  # 平均池化
        
        # 分類
        logits = self.classifier(pooled)
        
        return logits, attentions

# 建立模型
model = TransformerClassifier(
    vocab_size=vocab_size,
    d_model=32,
    num_heads=4,
    d_ff=64,
    num_layers=2,
    num_classes=2,
    max_len=50
).to(device)

print(model)
print(f"\n參數量: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 訓練

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
history = {'train_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(num_epochs):
    # 訓練
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        logits, _ = model(input_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    train_acc = 100 * correct / total
    history['train_loss'].append(total_loss / len(train_loader))
    history['train_acc'].append(train_acc)
    
    # 測試
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            
            logits, _ = model(input_ids)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    test_acc = 100 * correct / total
    history['test_acc'].append(test_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Loss: {history['train_loss'][-1]:.4f}, "
              f"Train Acc: {train_acc:.1f}%, Test Acc: {test_acc:.1f}%")

In [ ]:
# 視覺化訓練過程

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'])
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True)

axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['test_acc'], label='Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 視覺化 Attention

def visualize_attention(text, model, word2idx):
    model.eval()
    words = text.lower().split()
    tokens = [word2idx.get(w, 1) for w in words]
    
    # Padding
    tokens = tokens + [0] * (20 - len(tokens))
    input_ids = torch.tensor([tokens]).to(device)
    
    with torch.no_grad():
        logits, attentions = model(input_ids)
    
    prob = F.softmax(logits, dim=1)
    pred = "Positive" if prob[0, 1] > 0.5 else "Negative"
    
    # 取最後一層的 attention
    attn = attentions[-1][0].cpu().numpy()  # (num_heads, seq, seq)
    
    # 平均所有 head
    attn_avg = attn.mean(axis=0)[:len(words), :len(words)]
    
    plt.figure(figsize=(10, 8))
    plt.imshow(attn_avg, cmap='Blues')
    plt.colorbar()
    plt.xticks(range(len(words)), words, rotation=45)
    plt.yticks(range(len(words)), words)
    plt.xlabel('Key')
    plt.ylabel('Query')
    plt.title(f'Self-Attention (Pred: {pred}, Conf: {prob[0, 1 if pred=="Positive" else 0]:.2%})')
    plt.tight_layout()
    plt.show()

# 測試
visualize_attention("This movie is great and amazing", model, word2idx)
visualize_attention("This movie is terrible and boring", model, word2idx)

---

## Part 7：BERT 與 GPT 簡介

### 7.1 BERT (Bidirectional Encoder Representations from Transformers)

**核心特點：**
- 只用 Transformer **Encoder**
- **雙向**：可以同時看到左右上下文
- 預訓練任務：Masked Language Model (MLM) + Next Sentence Prediction (NSP)

**適用任務：** 分類、問答、命名實體識別等「理解」任務

In [ ]:
# BERT 的 Masked Language Model 示意

print("""
BERT 的預訓練任務 - Masked Language Model (MLM)：

輸入：  "The [MASK] sat on the [MASK]"
目標：       "cat"              "mat"

過程：
1. 隨機遮蔽 15% 的 token
2. 讓模型預測被遮蔽的 token
3. 因為可以看到兩邊的上下文，所以是「雙向」的

BERT 結構：
┌─────────────────────────────────────────────┐
│                                             │
│   [CLS] The [MASK] sat on the [MASK] [SEP]  │
│     ↓    ↓    ↓    ↓   ↓  ↓   ↓     ↓      │
│   ┌─────────────────────────────────────┐   │
│   │      Transformer Encoder x 12       │   │
│   └─────────────────────────────────────┘   │
│     ↓    ↓    ↓    ↓   ↓  ↓   ↓     ↓      │
│    分類  ...  cat  ... ...  mat  ...       │
│                                             │
└─────────────────────────────────────────────┘

[CLS]: 用於分類任務的特殊 token
[SEP]: 句子分隔符
""")

### 7.2 GPT (Generative Pre-trained Transformer)

**核心特點：**
- 只用 Transformer **Decoder**（有 causal mask）
- **單向**：只能看到左邊的上下文
- 預訓練任務：Next Token Prediction（自回歸語言模型）

**適用任務：** 文本生成、對話、寫作等「生成」任務

In [ ]:
# GPT 的自回歸生成示意

print("""
GPT 的預訓練任務 - Next Token Prediction：

輸入：  "The cat"
目標：       "sat"

生成過程（逐個 token 生成）：
"The" → "cat"
"The cat" → "sat"
"The cat sat" → "on"
...

GPT 結構：
┌─────────────────────────────────────────────┐
│                                             │
│   The    cat    sat    on    the    mat     │
│    ↓      ↓      ↓      ↓     ↓      ↓      │
│   ┌─────────────────────────────────────┐   │
│   │ Transformer Decoder x 12 (masked)  │   │
│   └─────────────────────────────────────┘   │
│    ↓      ↓      ↓      ↓     ↓      ↓      │
│   cat    sat    on    the   mat    .       │  ← 預測下一個 token
│                                             │
└─────────────────────────────────────────────┘

關鍵：使用 Causal Mask，每個位置只能看到之前的 token
""")

In [ ]:
# BERT vs GPT 比較

print("""
┌────────────────┬─────────────────────┬─────────────────────┐
│                │        BERT         │        GPT          │
├────────────────┼─────────────────────┼─────────────────────┤
│ 架構           │ Encoder-only        │ Decoder-only        │
│ 方向           │ 雙向 (Bidirectional)│ 單向 (Unidirectional)│
│ 預訓練         │ MLM + NSP           │ Next Token Predict  │
│ Attention Mask │ 無 (全部可見)       │ Causal (只看過去)   │
│ 主要用途       │ 理解任務            │ 生成任務            │
│ 代表模型       │ BERT, RoBERTa       │ GPT-2/3/4, ChatGPT  │
└────────────────┴─────────────────────┴─────────────────────┘
""")

---

## 練習題（已完成，請閱讀理解）

### 練習 1：使用 PyTorch 內建的 Transformer

**目標：** 學會使用 `nn.TransformerEncoder`

**Hint：** PyTorch 提供了優化過的 Transformer 實現

In [ ]:
# 練習 1：使用 PyTorch 內建 Transformer

# 定義 Transformer Encoder Layer
encoder_layer = nn.TransformerEncoderLayer(
    d_model=64,
    nhead=8,
    dim_feedforward=256,
    dropout=0.1,
    batch_first=True  # 重要！讓輸入格式是 (batch, seq, feature)
)

# 堆疊多層
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)

# 測試
x = torch.randn(2, 10, 64)  # (batch, seq_len, d_model)
output = transformer_encoder(x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {output.shape}")

# 帶 mask
mask = torch.triu(torch.ones(10, 10) * float('-inf'), diagonal=1)
output_masked = transformer_encoder(x, mask=mask)
print(f"\n帶 Causal Mask 的輸出 shape: {output_masked.shape}")

### 練習 2：不同 Head 數量的影響

**目標：** 觀察 Multi-Head 數量對模型的影響

**Hint：** head 數量需要能整除 d_model

In [ ]:
# 練習 2：比較不同 head 數量

d_model = 64
head_configs = [1, 2, 4, 8]

print("不同 head 數量的參數量比較：")
print("-" * 50)

for num_heads in head_configs:
    mha = MultiHeadAttention(d_model, num_heads)
    params = sum(p.numel() for p in mha.parameters())
    d_k = d_model // num_heads
    print(f"num_heads={num_heads}, d_k={d_k}, 參數量={params:,}")

print("\n觀察：參數量相同！因為總是 4 個 (d_model, d_model) 的矩陣")
print("區別在於：多個 head 可以學習不同的 attention pattern")

### 練習 3：Learned Positional Encoding

**目標：** 比較固定的正弦位置編碼和可學習的位置編碼

**Hint：** BERT 和 GPT 都使用 learned positional embedding

In [ ]:
# 練習 3：Learned Positional Embedding

class LearnedPositionalEncoding(nn.Module):
    """可學習的位置編碼（像 BERT/GPT 那樣）"""
    def __init__(self, d_model, max_len=512, dropout=0.1):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        seq_len = x.size(1)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0)
        pos_emb = self.pos_embedding(positions)
        return self.dropout(x + pos_emb)

# 比較兩種方式
d_model = 64
max_len = 100

sinusoidal_pe = PositionalEncoding(d_model, max_len, dropout=0)
learned_pe = LearnedPositionalEncoding(d_model, max_len, dropout=0)

print("位置編碼方式比較：")
print(f"固定（正弦餘弦）: 0 參數（預計算）")
print(f"可學習: {max_len * d_model:,} 參數")

print("\n優缺點：")
print("- 固定：可以處理比訓練時更長的序列，不需要額外參數")
print("- 可學習：更靈活，可能學到更好的位置表示，但有最大長度限制")

---

## Part 8：進階 Transformer 技巧

### 8.1 Cross-Attention（交叉注意力）

**用途：** Decoder 關注 Encoder 的輸出（用於翻譯、摘要等 seq2seq 任務）

**區別：**
- Self-Attention：Q, K, V 都來自同一序列
- Cross-Attention：Q 來自 Decoder，K 和 V 來自 Encoder

---

## Module 5 總結

### 核心概念

1. **Attention 機制**：
   - Query, Key, Value 三要素
   - 計算：score → softmax → weighted sum
   - 解決 RNN 的瓶頸問題

2. **Self-Attention**：
   - Q, K, V 來自同一個序列
   - 每個位置可以直接關注其他所有位置
   - Scaled Dot-Product：除以 √d_k

3. **Multi-Head Attention**：
   - 多個 head 捕捉不同類型的關係
   - 並行計算，效率高

4. **Positional Encoding**：
   - 正弦餘弦編碼（固定）
   - Learned Embedding（可學習）

5. **Transformer 結構**：
   - Encoder：Multi-Head Attention + FFN + 殘差 + LayerNorm
   - Decoder：額外加 Cross-Attention 和 Causal Mask

6. **預訓練模型**：
   - BERT：雙向 Encoder，適合理解任務
   - GPT：單向 Decoder，適合生成任務

### 下一步：Module 6 - LLM 實務入門

In [ ]:
# Cross-Attention 實現

class CrossAttention(nn.Module):
    """交叉注意力：Query 來自一個序列，Key/Value 來自另一個序列"""
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
    
    def forward(self, query_seq, kv_seq, mask=None):
        """
        query_seq: (batch, seq_len_q, d_model) - Decoder 的表示
        kv_seq: (batch, seq_len_kv, d_model) - Encoder 的輸出
        """
        # Query 來自 query_seq，Key 和 Value 來自 kv_seq
        return self.mha(query_seq, kv_seq, kv_seq, mask)

# 完整的 Transformer Decoder Layer
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        # Masked Self-Attention（看不到未來）
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        # Cross-Attention（關注 Encoder 輸出）
        self.cross_attention = MultiHeadAttention(d_model, num_heads)
        # Feed Forward
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        # 1. Masked Self-Attention
        self_attn, _ = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(self_attn))
        
        # 2. Cross-Attention（關注 Encoder）
        cross_attn, _ = self.cross_attention(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + self.dropout(cross_attn))
        
        # 3. Feed Forward
        ff = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff))
        
        return x

# 測試
decoder_layer = TransformerDecoderLayer(d_model=64, num_heads=8, d_ff=256)

# Decoder 輸入和 Encoder 輸出
decoder_input = torch.randn(2, 8, 64)   # (batch, tgt_len, d_model)
encoder_output = torch.randn(2, 12, 64) # (batch, src_len, d_model)

# Causal mask for decoder
tgt_mask = create_causal_mask(8).unsqueeze(0)

output = decoder_layer(decoder_input, encoder_output, tgt_mask=tgt_mask)
print(f"Decoder 輸入: {decoder_input.shape}")
print(f"Encoder 輸出: {encoder_output.shape}")
print(f"Decoder Layer 輸出: {output.shape}")

### 8.2 Pre-norm vs Post-norm

**Post-norm（原始 Transformer）：**
```
x → Attention → Add → LayerNorm → FFN → Add → LayerNorm
```

**Pre-norm（現代常用）：**
```
x → LayerNorm → Attention → Add → LayerNorm → FFN → Add
```

**Pre-norm 優點：** 訓練更穩定，尤其對深層網路

In [ ]:
# Pre-norm vs Post-norm 比較

class PreNormEncoderLayer(nn.Module):
    """Pre-norm 版本的 Encoder Layer（現代 LLM 常用）"""
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Pre-norm: 先 normalize，再做操作
        # Attention
        normed = self.norm1(x)
        attn_output, _ = self.self_attention(normed, normed, normed, mask)
        x = x + self.dropout(attn_output)
        
        # FFN
        normed = self.norm2(x)
        ff_output = self.feed_forward(normed)
        x = x + self.dropout(ff_output)
        
        return x

print("""
Pre-norm vs Post-norm 比較：

┌────────────────┬─────────────────┬─────────────────┐
│                │    Post-norm    │    Pre-norm     │
├────────────────┼─────────────────┼─────────────────┤
│ LayerNorm 位置 │ 子層之後        │ 子層之前        │
│ 訓練穩定性     │ 較差（深層）    │ 較好            │
│ 需要 warmup    │ 是              │ 通常不需要      │
│ 最終層 norm    │ 不需要          │ 需要額外 norm   │
│ 使用模型       │ BERT, 原始 GPT  │ GPT-2+, LLaMA   │
└────────────────┴─────────────────┴─────────────────┘

為什麼 Pre-norm 更穩定？
- 殘差連接直接連到輸出，梯度流更順暢
- LayerNorm 在前面，控制輸入範圍
- 對初始化不太敏感
""")

### 8.3 Transformer 訓練技巧

**重要的超參數和技巧：**

In [ ]:
# Transformer 訓練最佳實踐

print("""
╔═══════════════════════════════════════════════════════════════════╗
║                    Transformer 訓練技巧                           ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  1. 學習率調度（非常重要）                                         ║
║     ─────────────────────                                         ║
║     • Warmup + Decay 是標準做法                                   ║
║     • 原始論文：warmup 4000 steps，然後 inverse sqrt decay        ║
║     • 現代做法：cosine annealing 或 linear decay                  ║
║                                                                   ║
║  2. 權重初始化                                                    ║
║     ─────────────                                                 ║
║     • Xavier/Glorot 初始化（PyTorch 預設）                        ║
║     • 某些實現用 N(0, 0.02) 初始化 embedding                       ║
║     • 殘差連接前的層可以縮放：weight * (1/√num_layers)            ║
║                                                                   ║
║  3. 正則化                                                        ║
║     ──────                                                        ║
║     • Dropout: 0.1 是常見值                                       ║
║     • Label Smoothing: 0.1 可以提升泛化                           ║
║     • Weight Decay: AdamW 配合 0.01-0.1                           ║
║                                                                   ║
║  4. 梯度裁剪                                                      ║
║     ──────                                                        ║
║     • max_norm=1.0 是常見選擇                                     ║
║     • 對於大型模型更重要                                          ║
║                                                                   ║
║  5. Batch Size                                                    ║
║     ──────────                                                    ║
║     • 大 batch 配合 warmup 效果更好                               ║
║     • 若 GPU 記憶體不足，用 gradient accumulation                  ║
║                                                                   ║
╚═══════════════════════════════════════════════════════════════════╝
""")

# Warmup + Cosine Decay 學習率調度器
class TransformerLRScheduler:
    """帶 Warmup 的學習率調度"""
    def __init__(self, optimizer, d_model, warmup_steps):
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.step_num = 0
    
    def step(self):
        self.step_num += 1
        lr = self._get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
    
    def _get_lr(self):
        # 原始 Transformer 的學習率公式
        step = self.step_num
        return self.d_model ** (-0.5) * min(
            step ** (-0.5),
            step * self.warmup_steps ** (-1.5)
        )

# 視覺化學習率
warmup_steps = 4000
d_model = 512

steps = list(range(1, 50000))
scheduler_dummy = TransformerLRScheduler(None, d_model, warmup_steps)

lrs = []
for step in steps:
    scheduler_dummy.step_num = step
    lrs.append(scheduler_dummy._get_lr())

plt.figure(figsize=(10, 5))
plt.plot(steps, lrs)
plt.axvline(x=warmup_steps, color='r', linestyle='--', label='End of warmup')
plt.xlabel('Training Step')
plt.ylabel('Learning Rate')
plt.title('Transformer Learning Rate Schedule (Warmup + Inverse Sqrt Decay)')
plt.legend()
plt.grid(True)
plt.show()

print(f"Peak LR (at warmup_steps={warmup_steps}): {max(lrs):.6f}")

### 8.4 Attention 效率改進

**標準 Self-Attention 的問題：** 
- 時間複雜度：$O(n^2 \cdot d)$
- 空間複雜度：$O(n^2)$
- 長序列（如 n=4096）會非常慢且耗記憶體

In [ ]:
# Attention 複雜度分析

print("""
高效 Attention 變體：

╔════════════════════════════════════════════════════════════════════════╗
║  方法                    │ 複雜度       │ 說明                         ║
╠════════════════════════════════════════════════════════════════════════╣
║  Standard Attention      │ O(n²)       │ 原始版本                     ║
║  Sparse Attention        │ O(n√n)      │ 只關注部分位置（GPT-3）      ║
║  Linear Attention        │ O(n)        │ 改變計算順序（理論上）       ║
║  Flash Attention         │ O(n²)       │ IO-aware，實際上更快         ║
║  Multi-Query Attention   │ O(n²)       │ K,V 共享，減少記憶體         ║
║  Grouped-Query Attention │ O(n²)       │ MQA 和 MHA 的折衷            ║
╚════════════════════════════════════════════════════════════════════════╝

Flash Attention（PyTorch 2.0+）：
- 不改變數學計算，只優化 GPU 記憶體訪問
- 顯著減少記憶體使用和提升速度
- 使用方式：torch.nn.functional.scaled_dot_product_attention()
""")

# 比較標準 attention 和 PyTorch 2.0 的 flash attention
import time

def benchmark_attention(seq_len, d_model, num_heads, device):
    """比較不同 attention 實現的速度"""
    batch_size = 8
    
    q = torch.randn(batch_size, num_heads, seq_len, d_model // num_heads, device=device)
    k = torch.randn(batch_size, num_heads, seq_len, d_model // num_heads, device=device)
    v = torch.randn(batch_size, num_heads, seq_len, d_model // num_heads, device=device)
    
    # 標準實現
    def standard_attention(q, k, v):
        d_k = q.size(-1)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
        weights = F.softmax(scores, dim=-1)
        return torch.matmul(weights, v)
    
    # Warmup
    for _ in range(3):
        _ = standard_attention(q, k, v)
        _ = F.scaled_dot_product_attention(q, k, v)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # 測試標準版本
    start = time.time()
    for _ in range(10):
        _ = standard_attention(q, k, v)
    if device.type == 'cuda':
        torch.cuda.synchronize()
    standard_time = (time.time() - start) / 10
    
    # 測試 PyTorch 2.0+ 優化版本
    start = time.time()
    for _ in range(10):
        _ = F.scaled_dot_product_attention(q, k, v)
    if device.type == 'cuda':
        torch.cuda.synchronize()
    optimized_time = (time.time() - start) / 10
    
    return standard_time * 1000, optimized_time * 1000

print("\nAttention 速度比較（ms, 越低越好）：")
print("-" * 60)

for seq_len in [128, 256, 512]:
    std_time, opt_time = benchmark_attention(seq_len, 256, 8, device)
    speedup = std_time / opt_time
    print(f"seq_len={seq_len:4d}: 標準={std_time:.2f}ms, 優化={opt_time:.2f}ms, 加速={speedup:.1f}x")

print("\n提示：在 GPU 上且序列長度較大時，加速效果更明顯")

---

## Module 5 完整總結

### 🎯 核心概念

| 概念 | 說明 | 重要程度 |
|------|------|----------|
| **Attention** | Q·K^T 計算相關性，V 加權求和 | ⭐⭐⭐⭐⭐ |
| **Self-Attention** | Q, K, V 來自同一序列 | ⭐⭐⭐⭐⭐ |
| **Multi-Head** | 多個 head 捕捉不同關係 | ⭐⭐⭐⭐⭐ |
| **Positional Encoding** | 加入位置資訊 | ⭐⭐⭐⭐ |
| **Layer Normalization** | 穩定訓練 | ⭐⭐⭐⭐ |
| **殘差連接** | 梯度流動，深層訓練 | ⭐⭐⭐⭐⭐ |

### 🔑 關鍵公式

```
Scaled Dot-Product Attention:
  Attention(Q, K, V) = softmax(QK^T / √d_k) V

Multi-Head Attention:
  head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)
  MultiHead = Concat(head_1, ..., head_h)W^O

Positional Encoding:
  PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
  PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
```

### 🏗️ 架構對比

| 模型 | 架構 | 方向 | 預訓練任務 | 用途 |
|------|------|------|------------|------|
| **BERT** | Encoder | 雙向 | MLM + NSP | 理解任務 |
| **GPT** | Decoder | 單向 | Next Token | 生成任務 |
| **T5** | Enc-Dec | 混合 | Span Corruption | 通用任務 |

### 💡 實務技巧

1. **學習率**：
   - 必須用 Warmup（4000 steps 或 5-10% of total）
   - Warmup + Cosine/Linear Decay
   
2. **模型配置**：
   - d_model: 256/512/768/1024
   - num_heads: 8/12/16（確保 d_model % num_heads == 0）
   - d_ff = 4 * d_model（標準配置）
   - num_layers: 6/12/24

3. **正則化**：
   - Dropout: 0.1 是標準值
   - Label Smoothing: 0.1 可提升泛化
   
4. **效率優化**：
   - 使用 `F.scaled_dot_product_attention()`（Flash Attention）
   - 混合精度訓練（AMP）
   - Gradient Checkpointing（省記憶體）

5. **調試技巧**：
   - 視覺化 Attention Weights
   - 檢查不同 head 的模式
   - 監控梯度範數

### 🚀 下一步

- Module 6：LLM 實務（使用 Hugging Face）
- 進一步學習：Vision Transformer (ViT)、Multimodal Transformer

---

**恭喜完成 Transformer 模組！** 🎉

Transformer 是現代深度學習最重要的架構，從 NLP 到 CV 再到多模態，幾乎所有 SOTA 模型都基於 Transformer。掌握這個模組的內容，你就具備了理解 GPT、BERT、LLaMA 等大型語言模型的基礎。